In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import MinMaxScaler
import sklearn.metrics as skmet
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor

In [2]:
data = pd.read_csv("../data/original/posts.csv")

In [3]:
metricas = ['likes', 'love', 'angry', 'wow', 'haha', 'sad', 'shares']

In [4]:
data["reacciones"] = data[metricas].sum(1)

In [5]:
fdata=data[(data["scope"]!=0)&(data["reacciones"]>10)&(data["reacciones"]<=data["scope"])]

In [6]:
fdata.columns

Index(['id', 'url', 'idPagina', 'nombrePagina', 'likesPagina', 'idPost',
       'likes', 'love', 'angry', 'wow', 'haha', 'sad', 'shares', 'comments',
       'impresions', 'scope', 'postDate', 'views', 'tipo', 'paidStatus',
       'postNoPauta', 'reacciones'],
      dtype='object')

In [7]:
mdata = fdata[metricas+["scope","likesPagina","tipo"]]

In [8]:
mdata[:3]

,likes,love,angry,wow,haha,sad,shares,scope,likesPagina,tipo
0,18,7,0,0,0,0,4,3660,604830,Post
1,526,117,189,15,51,8,107,77468,604830,Video
2,28,1,0,0,0,0,13,4399,604830,Video


In [9]:
mdata["Post"] = mdata["tipo"].replace({"Post":1,"Video":0})
mdata["Video"] = mdata["tipo"].replace({"Post":0,"Video":1})

D:\Apps\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
D:\Apps\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
mdata[:3]

,likes,love,angry,wow,haha,sad,shares,scope,likesPagina,tipo,Post,Video
0,18,7,0,0,0,0,4,3660,604830,Post,1,0
1,526,117,189,15,51,8,107,77468,604830,Video,0,1
2,28,1,0,0,0,0,13,4399,604830,Video,0,1


In [11]:
transformer = FunctionTransformer(np.log1p, validate=True)
transformer_inv = FunctionTransformer(np.expm1, validate=True)

In [12]:
def loger(data):
    return transformer.transform(data)

In [13]:
def loger_back(data):
    return transformer_inv.transform(data)

In [14]:
mtdata = pd.concat([np.log1p(mdata[metricas + ["likesPagina","scope"]]),mdata[["Post","Video"]]],axis=1)

In [15]:
mtdata[:3]

,likes,love,angry,wow,haha,sad,shares,likesPagina,scope,Post,Video
0,2.944439,2.079442,0.000000,0.000000,0.000000,0.000000,1.609438,13.312704,8.205492,1,0
1,6.267201,4.770685,5.247024,2.772589,3.951244,2.197225,4.682131,13.312704,11.257633,0,1
2,3.367296,0.693147,0.000000,0.000000,0.000000,0.000000,2.639057,13.312704,8.389360,0,1


In [16]:
train_set, test_set = train_test_split(mtdata,test_size=0.2,random_state=42)

In [17]:
columnas = metricas + ["likesPagina"] + ["Post","Video"]
columnas = metricas
train_set_var = train_set[columnas].copy()
train_set_pred = train_set[["scope"]].copy()
test_set_var = test_set[columnas].copy()
test_set_pred = test_set[["scope"]].copy()

In [18]:
train_set_var[:3]

,likes,love,angry,wow,haha,sad,shares
5237,2.564949,0.000000,0.000000,0.000000,2.397895,0.693147,2.639057
978,4.574711,3.367296,1.945910,1.386294,1.386294,0.000000,4.653960
810,5.525453,3.951244,1.386294,1.945910,1.386294,0.000000,4.143135


In [19]:
train_set_pred[:3]

,scope
5237,7.633854
978,10.178996
810,10.224665


In [20]:
real_train = train_set_pred
real = np.expm1(real_train)

In [21]:
real_train[:3]

,scope
5237,7.633854
978,10.178996
810,10.224665


In [22]:
real_test = test_set_pred
realt = np.expm1(real_test)

In [23]:
clf = MLPRegressor(alpha=0.01, hidden_layer_sizes = (10,), max_iter = 50000, 
                 activation = 'logistic', learning_rate = 'adaptive',solver= 'lbfgs')

In [24]:
clf.fit(train_set_var,train_set_pred["scope"])
#clf.fit(train_set_var[metricas],train_set_pred["scope"])

MLPRegressor(activation='logistic', alpha=0.01, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(10,), learning_rate='adaptive',
             learning_rate_init=0.001, max_iter=50000, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=None, shuffle=True, solver='lbfgs', tol=0.0001,
             validation_fraction=0.1, verbose=False, warm_start=False)

In [26]:
train_set_var[:3]

,likes,love,angry,wow,haha,sad,shares
5237,2.564949,0.000000,0.000000,0.000000,2.397895,0.693147,2.639057
978,4.574711,3.367296,1.945910,1.386294,1.386294,0.000000,4.653960
810,5.525453,3.951244,1.386294,1.945910,1.386294,0.000000,4.143135


In [41]:
test = train_set_var.loc[0].values
test

array([2.94443898, 2.07944154, 0.        , 0.        , 0.        ,
       0.        , 1.60943791])

In [42]:
clf.predict([[2.94443898, 2.07944154, 0.        , 0.        , 0.        ,
       0.        , 1.60943791]])

array([8.09218004])

In [29]:
red_pred = pd.DataFrame(clf.predict(train_set_var),columns=["Prediccion"])
#red_pred = clf.predict(train_set_var[metricas])
red = np.expm1(red_pred)
#red = pd.DataFrame(red_pred)

In [30]:
red_pred[:3]

,Prediccion
0,8.157780
1,10.105851
2,10.386452


In [27]:
red[:3]

,Prediccion
0,2573.735212
1,23993.314686
2,31976.978174


In [28]:
red_test = pd.DataFrame(clf.predict(test_set_var),columns=["Prediccion"])
#red_test = clf.predict(test_set_var[metricas])
redt = np.expm1(red_test)
#redt = pd.DataFrame(red_test)

In [31]:
modelo = red_pred
modelot = red_test
modelol = red
modelolt = redt
nombre = "Red Neuronal Profunda"
exvatr = skmet.explained_variance_score(real_train,modelo)
exvate = skmet.explained_variance_score(real_test,modelot)
msqeltr = np.sqrt(skmet.mean_squared_error(real_train,modelo))
msqelte = np.sqrt(skmet.mean_squared_error(real_test,modelot))
msqetr = np.sqrt(skmet.mean_squared_error(real,modelol))
msqete = np.sqrt(skmet.mean_squared_error(realt,modelolt))
salida = pd.DataFrame({"real":real["scope"],"modelo":modelol["Prediccion"]})
maxetr = np.sqrt(np.power(salida["real"] - salida["modelo"],2).max())
salidat = pd.DataFrame({"real":realt["scope"],"modelo":modelolt["Prediccion"]})
maxete = np.sqrt(np.power(salidat["real"] - salidat["modelo"],2).max())

In [32]:
frame = pd.DataFrame({"Set de entrenamiento":[exvatr,msqeltr,msqetr,maxetr],"Set de prueba":[exvate,msqelte,msqete,maxete]},index=["Fracción de variabilidad explicada","Error cuadrático medio escala log","Error cuadratico medio","Error máximo"])
frame.index.name = nombre
frame.style.format("{:,}")

,Set de entrenamiento,Set de prueba
Red Neuronal Profunda,,
Fracción de variabilidad explicada,0.790484312762874,0.7950035180352744
Error cuadrático medio escala log,0.5926728770119154,0.6242708111222162
Error cuadratico medio,"79,917.45092646427","118,708.64698072789"
Error máximo,"6,651,495.82752593","2,869,922.0191242117"


In [33]:
scores = cross_val_score(clf,train_set_var,train_set_pred["scope"],scoring="neg_mean_squared_error",cv=10)

In [34]:
np.sqrt(scores*-1)

array([0.63639484, 0.64397098, 0.61738474, 0.74189676, 0.58032768,
       0.60140173, 0.60452276, 0.65064436, 0.61351134, 0.63920558])

In [35]:
param_grid = {"hidden_layer_sizes": [(10,),(30,),(10,10,)], "activation": ["logistic", "tanh", "relu"], "solver": ["lbfgs", "sgd", "adam"], "alpha": [0.001,0.005,0.01]}

In [36]:
grid_search = GridSearchCV(MLPRegressor(max_iter = 50000, learning_rate = 'adaptive'),param_grid,cv=5,scoring='neg_mean_squared_error',return_train_score=True,verbose=3,n_jobs=-1)

In [37]:
grid_search.fit(train_set_var,train_set_pred["scope"])

Fitting 5 folds for each of 108 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    9.6s
[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done 264 tasks      | elapsed: 15.3min
[Parallel(n_jobs=-1)]: Done 488 tasks      | elapsed: 22.0min
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed: 22.9min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=MLPRegressor(activation='relu', alpha=0.0001,
                                    batch_size='auto', beta_1=0.9, beta_2=0.999,
                                    early_stopping=False, epsilon=1e-08,
                                    hidden_layer_sizes=(100,),
                                    learning_rate='adaptive',
                                    learning_rate_init=0.001, max_iter=50000,
                                    momentum=0.9, n_iter_no_change=10,
                                    nesterovs_momentum=True, power_t=0.5,
                                    random_st...
                                    solver='adam', tol=0.0001,
                                    validation_fraction=0.1, verbose=False,
                                    warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'activation': ['logistic', 'tanh', 'relu'],
                         'alp

In [38]:
grid_search.best_params_

{'activation': 'tanh',
 'alpha': 0.01,
 'hidden_layer_sizes': (10,),
 'solver': 'lbfgs'}

In [39]:
cvres = grid_search.cv_results_

In [40]:
np.sort(np.array(list(zip(np.sqrt(cvres["mean_test_score"]*-1),cvres["params"])),dtype = [('error', float), ('params', 'S100')]),order="error")

array([(0.61323874, b"{'activation': 'tanh', 'alpha': 0.01, 'hidden_layer_sizes': (10,), 'solver': 'lbfgs'}"),
       (0.61350784, b"{'activation': 'logistic', 'alpha': 0.005, 'hidden_layer_sizes': (10,), 'solver': 'lbfgs'}"),
       (0.61548948, b"{'activation': 'tanh', 'alpha': 0.005, 'hidden_layer_sizes': (10,), 'solver': 'lbfgs'}"),
       (0.61654409, b"{'activation': 'logistic', 'alpha': 0.001, 'hidden_layer_sizes': (10,), 'solver': 'lbfgs'}"),
       (0.61681081, b"{'activation': 'logistic', 'alpha': 0.01, 'hidden_layer_sizes': (10,), 'solver': 'lbfgs'}"),
       (0.61692577, b"{'activation': 'relu', 'alpha': 0.001, 'hidden_layer_sizes': (30,), 'solver': 'lbfgs'}"),
       (0.61779525, b"{'activation': 'tanh', 'alpha': 0.001, 'hidden_layer_sizes': (10,), 'solver': 'lbfgs'}"),
       (0.61797719, b"{'activation': 'tanh', 'alpha': 0.01, 'hidden_layer_sizes': (10, 10), 'solver': 'adam'}"),
       (0.6192527 , b"{'activation': 'tanh', 'alpha': 0.01, 'hidden_layer_sizes': (30, 10), '

In [46]:
final_model = grid_search.best_estimator_

In [47]:
fmodel_pred = pd.DataFrame(final_model.predict(train_set_var),columns=["Prediccion"])
fmodel = np.expm1(fmodel_pred)

In [48]:
fmodel_test = pd.DataFrame(final_model.predict(test_set_var),columns=["Prediccion"])
fmodelt = np.expm1(fmodel_test)

In [49]:
modelo = fmodel_pred
modelot = fmodel_test
modelol = fmodel
modelolt = fmodelt
nombre = "Red Neuronal Profunda"
exvatr = skmet.explained_variance_score(real_train,modelo)
exvate = skmet.explained_variance_score(real_test,modelot)
msqeltr = np.sqrt(skmet.mean_squared_error(real_train,modelo))
msqelte = np.sqrt(skmet.mean_squared_error(real_test,modelot))
msqetr = np.sqrt(skmet.mean_squared_error(real,modelol))
msqete = np.sqrt(skmet.mean_squared_error(realt,modelolt))
salida = pd.DataFrame({"real":real["scope"],"modelo":modelol["Prediccion"]})
maxetr = np.sqrt(np.power(salida["real"] - salida["modelo"],2).max())
salidat = pd.DataFrame({"real":realt["scope"],"modelo":modelolt["Prediccion"]})
maxete = np.sqrt(np.power(salidat["real"] - salidat["modelo"],2).max())

In [50]:
frame = pd.DataFrame({"Set de entrenamiento":[exvatr,msqeltr,msqetr,maxetr],"Set de prueba":[exvate,msqelte,msqete,maxete]},index=["Fracción de variabilidad explicada","Error cuadrático medio escala log","Error cuadratico medio","Error máximo"])
frame.index.name = nombre
frame.style.format("{:,}")

,Set de entrenamiento,Set de prueba
Red Neuronal Profunda,,
Fracción de variabilidad explicada,0.786293038395448,0.7897233511024311
Error cuadrático medio escala log,0.5985716124217414,0.6322809354102273
Error cuadratico medio,"90,625.39985751086","82,067.63945992135"
Error máximo,"6,651,520.983932345","2,868,890.989917356"
